In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=449fae0351d10926dadcf1dab5cc900ba145f31e86bf513979f44ea07cc24965
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Conv1D, GlobalMaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Sequential




import seaborn as sns
import matplotlib.pyplot as plt

import re
import glob
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import warnings
from wordcloud import STOPWORDS, WordCloud
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/Datasets Sem-6/IR Datasets/Project/liar_dataset.zip'

Archive:  /content/drive/MyDrive/Datasets Sem-6/IR Datasets/Project/liar_dataset.zip
  inflating: README                  
  inflating: test.tsv                
  inflating: train.tsv               
  inflating: valid.tsv               


In [5]:
train_df = pd.read_csv('/content/train.tsv',sep='\t', header = None)
test_df = pd.read_csv('/content/test.tsv',sep='\t', header = None)
val_df = pd.read_csv('/content/valid.tsv',sep='\t', header = None)

In [6]:
train_df = train_df.drop([0, 8, 9, 10, 11, 12], axis = 1)
test_df = test_df.drop([0, 8, 9, 10, 11, 12], axis = 1)
val_df = val_df.drop([0, 8, 9, 10, 11, 12], axis = 1)

In [7]:
train_df.columns = ['label', 'statement', 'subject', 'speaker', 'speaker job title', 'state info', 'party affilation', 'location of statement']
test_df.columns = ['label', 'statement', 'subject', 'speaker', 'speaker job title', 'state info', 'party affilation', 'location of statement']
val_df.columns = ['label', 'statement', 'subject', 'speaker', 'speaker job title', 'state info', 'party affilation', 'location of statement']

In [8]:
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True)
test_df = test_df.dropna()
test_df = test_df.reset_index(drop=True)
val_df = val_df.dropna()
val_df = val_df.reset_index(drop=True)

In [9]:
train_df = pd.concat([train_df, val_df])
train_df = train_df.reset_index()

In [10]:
print('Training Data Dimensions -', train_df.shape)
print('Testing Data Dimensions -', test_df.shape)

Training Data Dimensions - (7585, 9)
Testing Data Dimensions - (853, 8)


In [11]:
labels_dict = {'mostly-true':4,'barely-true':2,'half-true':3,'false':1, 'true':5,'pants-fire':0}
train_df['label'] = train_df['label'].apply(lambda x: labels_dict[x])
test_df['label'] = test_df['label'].apply(lambda x: labels_dict[x])

In [12]:
def preprocess(text):
  text = text.lower() # lower - casing the text
  text = re.sub('<[^>]*>', ' ', text)
  text = re.sub('[\W]+', ' ', text)
  tokenizer = TreebankWordTokenizer()
  words = tokenizer.tokenize(text)
  text = ' '.join(words)
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [w for w in word_tokens if not w in stop_words] # removal of stopwords
  text = ' '.join(filtered_sentence)
  return text

In [13]:
# Standard Model['label' --> Target , 'statement' --> input text data]
def get_XY(data):
  y = data['label'].values
  X = data['statement'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

In [14]:
# Preprocessing text data
X_train, y_train = get_XY(train_df)
X_test, y_test = get_XY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))


Epoch 1/10
190/190 [==============================] - 15s 71ms/step - loss: -2072.8821 - accuracy: 0.1938 - val_loss: -10735.0938 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 16s 83ms/step - loss: -80415.0312 - accuracy: 0.1943 - val_loss: -208881.4062 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 9s 46ms/step - loss: -606200.1250 - accuracy: 0.1943 - val_loss: -1132892.5000 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 11s 60ms/step - loss: -2323729.0000 - accuracy: 0.1943 - val_loss: -3617297.5000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 11s 61ms/step - loss: -6202850.0000 - accuracy: 0.1943 - val_loss: -8660338.0000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 9s 45ms/step - loss: -13321475.0000 - accuracy: 0.1943 - val_loss: -17326170.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 11s 60m

In [15]:
# For Hybrid Model : Statement + Speaker
# Concatenate 'statement' and 'speaker' columns and provide it as input
def getHybridXY1(data): 
  y = data['label'].values
  X = data['statement'].values+data['speaker'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

X_train, y_train = getHybridXY1(train_df)
X_test, y_test = getHybridXY1(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))

Epoch 1/30
190/190 [==============================] - 16s 77ms/step - loss: -1558.3479 - accuracy: 0.1943 - val_loss: -8235.5586 - val_accuracy: 0.1984
Epoch 2/30
190/190 [==============================] - 14s 73ms/step - loss: -65936.7422 - accuracy: 0.1943 - val_loss: -174187.7656 - val_accuracy: 0.1984
Epoch 3/30
190/190 [==============================] - 13s 70ms/step - loss: -514772.1250 - accuracy: 0.1943 - val_loss: -966493.6875 - val_accuracy: 0.1984
Epoch 4/30
190/190 [==============================] - 13s 68ms/step - loss: -1985614.2500 - accuracy: 0.1943 - val_loss: -3105717.5000 - val_accuracy: 0.1984
Epoch 5/30
190/190 [==============================] - 10s 55ms/step - loss: -5315688.5000 - accuracy: 0.1943 - val_loss: -7430963.0000 - val_accuracy: 0.1984
Epoch 6/30
190/190 [==============================] - 13s 66ms/step - loss: -11490533.0000 - accuracy: 0.1943 - val_loss: -14950065.0000 - val_accuracy: 0.1984
Epoch 7/30
190/190 [==============================] - 13s 67m

In [16]:
# For Hybrid Model : Statement + Subject
# Concatenate 'statement' and 'subject' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['subject'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

  # Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))

Epoch 1/10
190/190 [==============================] - 13s 66ms/step - loss: -1123.4097 - accuracy: 0.1938 - val_loss: -6302.8076 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 11s 60ms/step - loss: -56889.8555 - accuracy: 0.1943 - val_loss: -150741.5625 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 14s 74ms/step - loss: -452400.5625 - accuracy: 0.1943 - val_loss: -851753.3750 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 11s 59ms/step - loss: -1755536.7500 - accuracy: 0.1943 - val_loss: -2750540.0000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 65ms/step - loss: -4730885.5000 - accuracy: 0.1943 - val_loss: -6651749.5000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 12s 63ms/step - loss: -10267682.0000 - accuracy: 0.1943 - val_loss: -13393136.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 10s 50m

In [17]:
# For Hybrid Model : Statement + speaker job title
# Concatenate 'statement' and 'speaker job title' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['speaker job title'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

  # Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))

Epoch 1/10
190/190 [==============================] - 14s 66ms/step - loss: -1061.1967 - accuracy: 0.1936 - val_loss: -5418.8311 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 10s 52ms/step - loss: -40651.2070 - accuracy: 0.1943 - val_loss: -104001.8984 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 14s 72ms/step - loss: -301070.6875 - accuracy: 0.1943 - val_loss: -554392.4375 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 13s 66ms/step - loss: -1122751.0000 - accuracy: 0.1943 - val_loss: -1737727.0000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 64ms/step - loss: -2975887.5000 - accuracy: 0.1943 - val_loss: -4133655.7500 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 11s 56ms/step - loss: -6388657.0000 - accuracy: 0.1943 - val_loss: -8275629.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 11s 60ms/

In [18]:
# For Hybrid Model : Statement + state info
# Concatenate 'statement' and 'state info' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['state info'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

  # Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))

Epoch 1/10
190/190 [==============================] - 12s 58ms/step - loss: -1698.0819 - accuracy: 0.1940 - val_loss: -8577.8633 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 15s 79ms/step - loss: -63744.8320 - accuracy: 0.1943 - val_loss: -162853.1406 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 12s 64ms/step - loss: -465517.1562 - accuracy: 0.1943 - val_loss: -859246.0000 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 10s 53ms/step - loss: -1738582.8750 - accuracy: 0.1943 - val_loss: -2691579.5000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 66ms/step - loss: -4581034.0000 - accuracy: 0.1943 - val_loss: -6388207.5000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 12s 65ms/step - loss: -9790500.0000 - accuracy: 0.1943 - val_loss: -12719079.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 11s 59ms

In [19]:
# For Hybrid Model : Statement + party affilation
# Concatenate 'statement' and 'party affilation' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['party affilation'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

  # Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))

Epoch 1/10
190/190 [==============================] - 13s 63ms/step - loss: -953.2339 - accuracy: 0.1935 - val_loss: -4612.2568 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 11s 57ms/step - loss: -31876.7656 - accuracy: 0.1943 - val_loss: -80221.2656 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 13s 69ms/step - loss: -225913.1719 - accuracy: 0.1943 - val_loss: -413576.8125 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 12s 62ms/step - loss: -836843.0000 - accuracy: 0.1943 - val_loss: -1290745.6250 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 9s 49ms/step - loss: -2190837.7500 - accuracy: 0.1943 - val_loss: -3043496.0000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 12s 62ms/step - loss: -4656988.5000 - accuracy: 0.1943 - val_loss: -6032833.5000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 12s 65ms/step

In [20]:
# For Hybrid Model : Statement + location of statement
# Concatenate 'statement' and 'location of statement' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['location of statement'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

  # Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluating the model on the test set
score, acc = model.evaluate(X_test, y_test)

print('Test score:', score)
print('Test accuracy:', acc)
test_preds = model.predict(X_test)
print(classification_report(y_test, test_preds))

Epoch 1/10
190/190 [==============================] - 16s 81ms/step - loss: -1943.7203 - accuracy: 0.1945 - val_loss: -10039.0088 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 11s 57ms/step - loss: -78095.5312 - accuracy: 0.1943 - val_loss: -198714.7031 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 11s 55ms/step - loss: -565913.7500 - accuracy: 0.1943 - val_loss: -1039818.5625 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 12s 64ms/step - loss: -2104802.7500 - accuracy: 0.1943 - val_loss: -3263417.7500 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 62ms/step - loss: -5573928.0000 - accuracy: 0.1943 - val_loss: -7769659.5000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 10s 50ms/step - loss: -11947067.0000 - accuracy: 0.1943 - val_loss: -15523917.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 12s 6